In [1]:
# Import necessary modules
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
import pickle
from tqdm import tqdm
from scipy import signal
import scipy
import pandas as pd
from pathlib import Path
import torch
import pathlib

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
data_dir = pathlib.Path("/data0/practical-sose23/brain-age/data/lemon/LEMON_RAW")

In [4]:
eeg_files_all = list(data_dir.rglob("*.vhdr"))
len(eeg_files_all)

215

In [5]:
f_namemap = list(data_dir.parent.rglob("*name*.csv"))[0]
df_namemap = pd.read_csv(f_namemap)
namemap = {old:new for old, new in zip(df_namemap["Initial_ID"], df_namemap["INDI_ID"])}

In [6]:
def get_midpoint(age_range):
    low, high = age_range.split("-")
    return (int(high)+int(low))/2

f_subj = list(data_dir.parent.rglob("*Age*.csv"))[0]
df_subj = pd.read_csv(f_subj)
df_subj["Subject ID"] = df_subj['Unnamed: 0']
df_subj, df_subj["Age"].apply(get_midpoint)

(     Unnamed: 0  Gender_ 1=female_2=male    Age Handedness  Education  \
 0    sub-032311                        2  20-25      right  Gymnasium   
 1    sub-032309                        2  20-25      right  Gymnasium   
 2    sub-032382                        2  25-30      right  Gymnasium   
 3    sub-032302                        1  20-25      right  Gymnasium   
 4    sub-032315                        2  20-25      right  Gymnasium   
 ..          ...                      ...    ...        ...        ...   
 223  sub-032304                        2  25-30      right  Gymnasium   
 224  sub-032439                        2  20-25       left  Gymnasium   
 225  sub-032440                        2  25-30       left  Gymnasium   
 226  sub-032441                        2  25-30      right  Gymnasium   
 227  sub-032525                        2  20-25      right  Gymnasium   
 
          DRUG  DRUG_0=negative_1=Positive Unnamed: 7            Smoking  \
 0    negative                    

In [7]:
def get_midpoint(age_range):
    low, high = age_range.split("-")
    return (int(high)+int(low))/2 

def get_subj_id(eeg_file: pathlib.Path):
    return eeg_file.stem

def get_age(df_subj, eeg_file: pathlib.Path):
    subj_id = get_subj_id(eeg_file)
    return get_midpoint(df_subj[df_subj["Subject ID"] == namemap[subj_id]]["Age"].values[0])

In [8]:
exceptions = []
eeg_len = []
for file_path in tqdm(eeg_files_all):
    try:
        example_raw = mne.io.read_raw_brainvision(file_path, preload=True)
        assert example_raw.get_data().shape[0] == 62
        eeg_len.append(example_raw.get_data().shape[1] / 2500)
    except Exception as e:
        exceptions.append((file_path, e))
        continue

  0%|                                                                                                     | 0/215 [00:00<?, ?it/s]

Extracting parameters from /data0/practical-sose23/brain-age/data/lemon/LEMON_RAW/sub-010270/RSEEG/sub-010270.vhdr...
Setting channel info structure...
Reading 0 ... 2550899  =      0.000 ...  1020.360 secs...


  0%|▍                                                                                            | 1/215 [00:01<06:48,  1.91s/it]

&&&&&&&&&&&&&&&:  62
Extracting parameters from /data0/practical-sose23/brain-age/data/lemon/LEMON_RAW/sub-010202/RSEEG/sub-010202.vhdr...
Setting channel info structure...
Reading 0 ... 2538099  =      0.000 ...  1015.240 secs...


  1%|▊                                                                                            | 2/215 [00:03<06:50,  1.93s/it]

...

Extracting parameters from /data0/practical-sose23/brain-age/data/lemon/LEMON_RAW/sub-010078/RSEEG/sub-010078.vhdr...
Setting channel info structure...
Reading 0 ... 2685599  =      0.000 ...  1074.240 secs...
&&&&&&&&&&&&&&&:  62


 97%|████████████████████████████████████████████████████████████████████████████████████████▍  | 209/215 [07:37<00:13,  2.23s/it]

Extracting parameters from /data0/practical-sose23/brain-age/data/lemon/LEMON_RAW/sub-010268/RSEEG/sub-010268.vhdr...
Setting channel info structure...
Reading 0 ... 2553099  =      0.000 ...  1021.240 secs...


 97%|████████████████████████████████████████████████████████████████████████████████████████▍  | 209/215 [07:39<00:13,  2.20s/it]


KeyboardInterrupt: 

In [9]:
print(min(eeg_len), max(eeg_len))

80.1 1263.56


In [ ]:
channel_to_channel_type = {ch_name:"eeg" for ch_name in example_raw.ch_names}
channel_to_channel_type.update({"VEOG":"misc"})
# eeg_chs = [ch for ch in example_raw.ch_names if not (ch=="RE" or ch=="LE")]
example_raw.set_channel_types(channel_to_channel_type)

In [ ]:
print(exceptions)

In [ ]:
montage = mne.channels.make_standard_montage("standard_1020")
example_raw = example_raw.set_montage(montage)

In [12]:
test_path = '/data0/practical-sose23/brain-age/data/lemon/preprocessed/v2.0/sub-010319/RSEEG/sub-010319/sub-010319_9.npy'
with open(test_path, 'rb') as in_file:
    npy_file = np.load(in_file)
print(npy_file.shape[1])

3000


In [ ]:
data_dir = pathlib.Path("/data0/practical-sose23/brain-age/data/lemon/preprocessed")
file_paths = list(data_dir.rglob("*.npy"))
print(len(file_paths))

In [ ]:
eeg_len = []

for file_path in tqdm(file_paths):
    with open(file_path, 'rb') as in_file:
        npy_file = np.load(in_file)
        eeg_len.append(npy_file.shape[1])

In [ ]:
sfreq = 100
print(min(eeg_len) / sfreq, max(eeg_len) / sfreq)